In [2]:
from binance.client import Client
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta

In [3]:
# Load .env
load_dotenv()

# Get API key
api_key = os.getenv('BINANCE_API_KEY')
api_secret = os.getenv('BINANCE_API_SECRET')

In [10]:
# Create a Binance client
client = Client(api_key, api_secret)

def fetch_binance_data(symbol='BTCUSDT', interval='1d'):
    """
    Get history OHLCV data for Bitcoin (BTC)
    :param symbol:  'BTCUSDT'
    :param interval: K-line period, '1d'
    :return: pandas DataFrame
    """
    # Calculate the date from three years ago and format it in 'YYYY-MM-DD UTC' format
    three_years_ago = datetime.now() - timedelta(days=3*365)
    start_date = three_years_ago.strftime('%Y-%m-%d')+ ' UTC'
    
    # Get historical data
    klines = client.get_historical_klines(symbol, interval, start_str=start_date)

    # Convert data to -- pandas DataFrame
    ohlcv_data = pd.DataFrame(klines, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume', 
        'close_time', 'quote_asset_volume', 'number_of_trades', 
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])

    # Converts the timestamp to date format
    ohlcv_data['timestamp'] = pd.to_datetime(ohlcv_data['timestamp'], unit='ms')

   # Add Unix timestamp (seconds since epoch) in scientific notation
    ohlcv_data['timestamp_unix'] = ohlcv_data['timestamp'].apply(lambda x: "{:.6e}".format(x.timestamp()))

    # return
    return ohlcv_data[['timestamp', 'timestamp_unix', 'open', 'high', 'low', 'close', 'volume']]


# Last three years of historical data
symbol = 'BTCUSDT' 
data = fetch_binance_data(symbol=symbol, interval='1d')

# output data -- 5
print(data.head())

# Save as CSV
output_filename = "bitcoin_3_years_data.csv"
data.to_csv(output_filename, index=False)

   timestamp timestamp_unix            open            high             low  \
0 2021-12-20   1.639958e+09  46681.24000000  47537.57000000  45558.85000000   
1 2021-12-21   1.640045e+09  46914.17000000  49328.96000000  46630.00000000   
2 2021-12-22   1.640131e+09  48887.59000000  49576.13000000  48421.87000000   
3 2021-12-23   1.640218e+09  48588.17000000  51375.00000000  47920.42000000   
4 2021-12-24   1.640304e+09  50838.82000000  51810.00000000  50384.43000000   

            close          volume  
0  46914.16000000  35848.50609000  
1  48889.88000000  37713.92924000  
2  48588.16000000  27004.20220000  
3  50838.81000000  35192.54046000  
4  50820.00000000  31684.84269000  
